<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/master/api_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xmltodict

In [2]:
import pandas as pd
import re
import sys
import requests
import json
import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen
import xmltodict
from tqdm import tqdm
import random
import pprint

In [3]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)
totalCnt = int(dict_contents['LawSearch']['totalCnt'])

In [8]:
totalCnt

2608

In [508]:
rows = []
not_match_jo = []
changed_num = 0
changed = []
eng_cnt = 0
kor_cnt = 0
total_page = int(totalCnt/20)+1
for p in tqdm(range(1,total_page)):
  url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=' + str(p)
  # url 불러오기
  response = requests.get(url)
  # XML -> 딕셔너리 변환후 파싱
  contents = response.text
  dict_contents = xmltodict.parse(contents)
  try:
    for content in dict_contents['LawSearch']['law']:
      name_kor = content['법령명한글']
      name_eng = content['법령명영문']
      mst = content['법령일련번호']

      eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      eng_response = requests.get(eng_url)
      eng_contents = eng_response.text
      eng_dict_contents = xmltodict.parse(eng_contents)

      kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      kor_response = requests.get(kor_url)
      kor_contents = kor_response.text
      kor_dict_contents = xmltodict.parse(kor_contents)

      # # 영어조문 개수와 한글 조문 개수 맞지 않다면 건너뛰기
      # if len(eng_dict_contents['Law']['JoSection']['Jo']) != len(kor_dict_contents['법령']['조문']['조문단위']):
      #     not_match_jo += 1
      #     print(f'영한 조문 개수 맞지 않는 법령 개수', not_match_jo, kor_dict_contents['법령']['기본정보']['법령명_한글'])
      #     continue
      # eng_cnt += len(eng_dict_contents['Law']['JoSection']['Jo'])
      # kor_cnt += len(kor_dict_contents['법령']['조문']['조문단위'])
      min_num_jo = min(len(eng_dict_contents['Law']['JoSection']['Jo']), len(kor_dict_contents['법령']['조문']['조문단위']))
      temp_cnt = 0
      print('법령명:', name_kor, ', 조문 개수:', min_num_jo, ', 페이지:', p, ', 몇번째?:', content['@id'])
      try:
        # for eng_jo, kor_jo in zip(eng_dict_contents['Law']['JoSection']['Jo'],kor_dict_contents['법령']['조문']['조문단위']):
        for ind in range(min_num_jo):
          eng_jo, kor_jo = eng_dict_contents['Law']['JoSection']['Jo'][ind], kor_dict_contents['법령']['조문']['조문단위'][ind]

          if kor_jo['조문여부']!='조문':
            continue
          # if kor_jo.get('조문가지번호'):
          #   continue

          kor_total = 0
          if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
            if type(kor_jo['항']) == dict:
              kor_total += len(kor_jo['항']['호'])
              for ho in kor_jo['항']['호']:
                if ho.get('목'):
                  kor_total += len(ho['목'])
              kor_total += 1 # 호부터 시작하면 영어에서는 또 \n이 하나 늘어남
            else:
              kor_total += len(kor_jo['항'])
              for hang in kor_jo['항']:
                if hang.get('호'):
                  kor_total += len(hang['호'])
                  for ho in hang['호']:
                    if ho.get('목'):
                      kor_total += len(ho['목'])
          else:
            kor_total = 1

          if kor_jo.get('조문참고자료'):
            if '\n\t\t\n\t\t' in kor_jo['조문참고자료']:
              kor_total += 2
              kor_jo['조문참고자료'] = kor_jo['조문참고자료'].replace('\n\t\t\n\t\t', '\n')
            else:
              kor_total += 1

          if (eng_jo['joCts'].count('\n')+1) != kor_total:
            not_match_jo.append([name_kor, kor_jo['조문제목']])
            continue

          if eng_jo['joYn']!= 'Y':
            changed_num += 1
            changed.append([eng_jo, kor_jo])
            print('안바뀐 법령 개수:', changed_num)
            continue
          # if eng_jo['joBrNo']!='00':
          #   continue

          eng_text = eng_jo['joCts']

          jo_txt = ''
          hang_txt = ''
          ho_txt = ''
          mok_txt = ''

          jo_txt += kor_jo['조문내용']
          if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
            if type(kor_jo['항']) == dict:
              for ho in kor_jo['항']['호']:
                ho_txt += ho['호내용']
                if ho.get('목'):
                  for mok in ho['목']:
                    mok_txt = mok['목내용']
                  ho_txt += mok_txt
              hang_txt += ho_txt
            else:
              for hang in kor_jo['항']:
                hang_txt += hang['항내용']
                if hang.get('호'):
                  for ho in hang['호']:
                    ho_txt += ho['호내용']
                    if ho.get('목'):
                      for mok in ho['목']:
                        mok_txt = mok['목내용']
                        ho_txt += mok_txt
                  hang_txt += ho_txt
              jo_txt += hang_txt
          if kor_jo.get('조문참고자료'):
            jo_txt += kor_jo['조문참고자료']
          kor_text = jo_txt
          rows.append({'name_kor':name_kor, 'name_eng':name_eng, 'MST':mst, 'eng':eng_text, 'kor':kor_text})
      except Exception as e:
        print('eng_kor_contents 예외 발생:', e)
  except Exception as e:
    print('dict_contents 예외 발생:', e)
print(len(not_match_jo))
df = pd.DataFrame(rows)

  0%|          | 0/130 [00:00<?, ?it/s]

법령명: 119구조ㆍ구급에 관한 법률 , 조문 개수: 47 , 페이지: 1 , 몇번째?: 1
법령명: 119구조ㆍ구급에 관한 법률 시행령 , 조문 개수: 47 , 페이지: 1 , 몇번째?: 2
법령명: 1인 창조기업 육성에 관한 법률 , 조문 개수: 24 , 페이지: 1 , 몇번째?: 3


  0%|          | 0/130 [00:23<?, ?it/s]


KeyboardInterrupt: ignored

In [501]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=1'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)

content = dict_contents['LawSearch']['law'][5]

name_kor = content['법령명한글']
name_eng = content['법령명영문']
mst = content['법령일련번호']
print(name_kor)
eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
          + '&MST=' + str(mst)
          # + '&ID=' + str(법령ID)
eng_response = requests.get(eng_url)
eng_contents = eng_response.text
eng_dict_contents = xmltodict.parse(eng_contents)

kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
          + '&MST=' + str(mst)
          # + '&ID=' + str(법령ID)
kor_response = requests.get(kor_url)
kor_contents = kor_response.text
kor_dict_contents = xmltodict.parse(kor_contents)

2018 평창 동계올림픽대회 및 동계패럴림픽대회 지원 등에 관한 특별법


In [502]:
count = 0
for temp in kor_dict_contents['법령']['조문']['조문단위']:
    if temp.get('조문제목'):
        if temp['조문제목'] == '기념화폐의 판매':
            print(count)
    count += 1

17


In [503]:
eng_dict_contents['Law']['JoSection']['Jo'][17]

{'@No': '18',
 'chapNo': '00000200000000000000',
 'joNo': '0014',
 'joBrNo': '00',
 'joYn': 'Y',
 'joTtl': 'Sale of Commemorative Banknotes',
 'joCts': 'Article 14 (Sale of Commemorative Banknotes) (1) In order to cover the expenses to be incurred in preparing and operating the Games, the Organizing Committee may request the Bank of Korea to issue commemorative banknotes. <Amended by Act No. 14198, May 29, 2016>\n  (2) The Organizing Committee may exclusively acquire commemorative banknotes issued under paragraph (1). <Amended by Act No. 14198, May 29, 2016>\n  [This Article remains in force through Mar. 31, 2019 pursuant to Article 2 of Addenda to Act No. 11226, Jan. 26, 2012]'}

In [504]:
kor_dict_contents['법령']['조문']['조문단위'][17]

{'@조문키': '0014001',
 '조문번호': '14',
 '조문여부': '조문',
 '조문제목': '기념화폐의 판매',
 '조문시행일자': '20190401',
 '조문이동이전': None,
 '조문이동이후': None,
 '조문변경여부': 'N',
 '조문내용': '제14조(기념화폐의 판매)',
 '항': [{'항번호': '①',
   '항내용': '① 조직위원회는 대회의 준비ㆍ운영에 필요한 비용을 충당하기 위하여 기념화폐의 발행을 한국은행에 요청할 수 있다. <개정 2016.5.29>'},
  {'항번호': '②',
   '항내용': '②조직위원회는 제1항에 따라 발행된 기념화폐를 독점적으로 인수할 수 있다. <개정 2016.5.29>'}],
 '조문참고자료': '[제목개정 2016.5.29]\n\t\t\n\t\t[법률 제11226호(2012. 1. 26.) 제14조의 개정규정은 같은 법 부칙 제2조의 규정에 의하여 2019년 3월 31일까지 유효함]'}

In [179]:
# 영한 조문 개수 맞지 않는 법령 개수 2 간접투자자산 운용업법 시행령 17
# 영한 조문 개수 맞지 않는 법령 개수 3 간척지의 농어업적 이용 및 관리에 관한 법률 9
# 영한 조문 개수 맞지 않는 법령 개수 4 개발이익 환수에 관한 법률 14
# 영한 조문 개수 맞지 않는 법령 개수 5 개별소비세법 18

In [485]:
kor_jo = kor_dict_contents['법령']['조문']['조문단위'][43]
eng_jo = eng_dict_contents['Law']['JoSection']['Jo'][43]
kor_total = 0
if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
  if type(kor_jo['항']) == dict:
    kor_total += len(kor_jo['항']['호'])
    for ho in kor_jo['항']['호']:
      if ho.get('목'):
        kor_total += len(ho['목'])
    kor_total += 1
  else:
    kor_total += len(kor_jo['항'])
    for hang in kor_jo['항']:
      if hang.get('호'):
        kor_total += len(hang['호'])
        for ho in hang['호']:
          if ho.get('목'):
            kor_total += len(ho['목'])
else:
  kor_total = 1
print(kor_total)

if kor_jo.get('조문참고자료'):
  kor_total += (1 + kor_jo['조문참고자료'].count('\n'))
print(kor_total)
if (eng_jo['joCts'].count('\n')+1) != kor_total:
  not_match_jo += 1
  print(not_match_jo)

6
9
4


In [487]:
eng_jo['joCts'].count('\n')
# kor_jo
# eng_jo

AttributeError: ignored